In [47]:
import pandas as pd
import numpy as np
from sklearn.model_selection import GridSearchCV, train_test_split, StratifiedKFold
from lightgbm import LGBMClassifier
import lightgbm
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix
import plotly.express as px

In [15]:
from sklearn.preprocessing import LabelEncoder


def convert_category_into_integer(df: pd.DataFrame, columns: list):

    label_encoders = {}  # 각 열의 LabelEncoder 객체를 저장할 딕셔너리입니다.
    
    for column in columns:
        # 각 열에 대해 LabelEncoder 객체를 생성합니다.
        label_encoder = LabelEncoder()
        
        # LabelEncoder를 사용하여 해당 열의 범주형 데이터를 정수형으로 변환합니다.
        df.loc[:, column] = label_encoder.fit_transform(df[column])
        
        # 변환된 LabelEncoder 객체를 딕셔너리에 저장합니다.
        label_encoders.update({column: label_encoder})
    
    # 변환된 데이터프레임과 LabelEncoder 객체를 포함하는 딕셔너리를 반환합니다.
    return df, label_encoders


In [16]:
seed = 0

In [17]:
data_train = pd.read_csv(r'C:\Users\USER\Desktop\MyCode_BH\SkAiCampMyData\project2\data\train.csv')
data_test = pd.read_csv(r'C:\Users\USER\Desktop\MyCode_BH\SkAiCampMyData\project2\data\test.csv')

In [18]:
data_train_dropna = data_train.dropna()

In [19]:
data_train_dropna_labeled, _ = convert_category_into_integer(data_train_dropna, ('Churn','ServiceArea','ChildrenInHH','HandsetRefurbished','HandsetWebCapable','TruckOwner','RVOwner','Homeownership','BuysViaMailOrder','RespondsToMailOffers','OptOutMailings','NonUSTravel','OwnsComputer','HasCreditCard','NewCellphoneUser','NotNewCellphoneUser','OwnsMotorcycle','HandsetPrice','MadeCallToRetentionTeam','CreditRating','PrizmCode','Occupation','MaritalStatus'))

In [21]:
data_train_dropna_labeled[['Churn','ServiceArea','ChildrenInHH','HandsetRefurbished','HandsetWebCapable','TruckOwner','RVOwner','Homeownership','BuysViaMailOrder','RespondsToMailOffers','OptOutMailings','NonUSTravel','OwnsComputer','HasCreditCard','NewCellphoneUser','NotNewCellphoneUser','OwnsMotorcycle','HandsetPrice','MadeCallToRetentionTeam','CreditRating','PrizmCode','Occupation','MaritalStatus']]=data_train_dropna_labeled[['Churn','ServiceArea','ChildrenInHH','HandsetRefurbished','HandsetWebCapable','TruckOwner','RVOwner','Homeownership','BuysViaMailOrder','RespondsToMailOffers','OptOutMailings','NonUSTravel','OwnsComputer','HasCreditCard','NewCellphoneUser','NotNewCellphoneUser','OwnsMotorcycle','HandsetPrice','MadeCallToRetentionTeam','CreditRating','PrizmCode','Occupation','MaritalStatus']].astype(int)

C:\Users\USER\AppData\Local\Temp\ipykernel_19024\3957736257.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_train_dropna_labeled[['Churn','ServiceArea','ChildrenInHH','HandsetRefurbished','HandsetWebCapable','TruckOwner','RVOwner','Homeownership','BuysViaMailOrder','RespondsToMailOffers','OptOutMailings','NonUSTravel','OwnsComputer','HasCreditCard','NewCellphoneUser','NotNewCellphoneUser','OwnsMotorcycle','HandsetPrice','MadeCallToRetentionTeam','CreditRating','PrizmCode','Occupation','MaritalStatus']]=data_train_dropna_labeled[['Churn','ServiceArea','ChildrenInHH','HandsetRefurbished','HandsetWebCapable','TruckOwner','RVOwner','Homeownership','BuysViaMailOrder','RespondsToMailOffers','OptOutMailings','NonUSTravel','OwnsComputer','HasCre

In [23]:
data_train_dropna_labeled = data_train_dropna_labeled.drop(columns=[
    'CustomerID',  # 구분용도임
    'NotNewCellphoneUser', # NewCell...이랑 중복
    'DroppedCalls', # DroppedBloaked가 있어서 삭제
    'BlockedCalls', 
    'PrizmCode', # 인구통계 세분화 코드라 삭제
    'ServiceArea', #지역정보라서 삭제
    'TruckOwner', # 자동차 오토바이 유무라서 삭제
    'RVOwner',
    'OwnsMotorcycle',
    'OwnsComputer', # 컴퓨터 유무 삭제
    'CurrentEquipmentDays', # 사용일수는 월별서비스와 중복
    'HandsetRefurbished', #
    'PeakCallsInOut', # 전화 시간대에 관련 삭제
    'OffPeakCallsInOut',
    'BuysViaMailOrder', # 우편관련 삭제
    'RespondsToMailOffers',
    'OptOutMailings',
    'NonUSTravel',# 미국 여행여부 삭제
    'AdjustmentsToCreditRating', # 신용등급
    'UniqueSubs', # 고유 구독자 수 활동 구독자 수 삭제
    'ActiveSubs',
])


In [26]:
a = data_train_dropna_labeled['UnansweredCalls'] > 80 # 고객이 80번이상 전화를 받지 않은건 이상하다.
data_train_dropna_labeled = data_train_dropna_labeled[~a]
a = data_train_dropna_labeled['CustomerCareCalls'] > 50 # 고객센터에 50번 이상 전화한건 잘못입력된것같다
data_train_dropna_labeled = data_train_dropna_labeled[~a]
a = data_train_dropna_labeled['ReceivedCalls'] > 500 # 전화를 받은수가 500 이상인건 이상하다.
data_train_dropna_labeled = data_train_dropna_labeled[~a]
a = data_train_dropna_labeled['OutboundCalls'] > 250 # 발신수가 250이상 삭제
data_train_dropna_labeled = data_train_dropna_labeled[~a]
a = data_train_dropna_labeled['DroppedBlockedCalls'] > 250 # 실패전화 250이상 삭제
data_train_dropna_labeled = data_train_dropna_labeled[~a]
a = data_train_dropna_labeled['CallForwardingCalls'] > 30 # 다른번호로 전화를 연결한 횟수
data_train_dropna_labeled = data_train_dropna_labeled[~a]
a = data_train_dropna_labeled['CallWaitingCalls'] > 30 # 통화 대기중 걸려온 전화 수
data_train_dropna_labeled = data_train_dropna_labeled[~a]

In [29]:
data_train_dropna_labeled.isna().sum().sum()

0

In [30]:
dt_t, dt_v = train_test_split(data_train_dropna_labeled, test_size=0.3, random_state=seed)

In [31]:
x= dt_t.drop(columns='Churn')
y = dt_t['Churn']

In [38]:
parm = {
    'num_iterations' : [50, 100],
    'max_depth' : [10,15,20],
    'reg_alpha' : [0.0,0.2,0.3,0.5,0.7, 0.9],
    'reg_lambda': [0.0,0.2,0.3,0.5,0.7, 0.9], 

}

In [39]:
lgbm = LGBMClassifier(random_state=seed)

In [40]:
grid_lgbm= GridSearchCV(lgbm, parm, cv=5)

In [41]:
grid_lgbm.fit(x, y)

c:\Users\USER\.conda\envs\deepLearning\lib\site-packages\lightgbm\engine.py:204: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")
c:\Users\USER\.conda\envs\deepLearning\lib\site-packages\lightgbm\engine.py:204: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


[LightGBM] [Info] Number of positive: 7309, number of negative: 17891
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.003848 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3416
[LightGBM] [Info] Number of data points in the train set: 25200, number of used features: 36
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.290040 -> initscore=-0.895191
[LightGBM] [Info] Start training from score -0.895191
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Info] Number of positive: 7310, number of negative: 17891
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.002460 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3407
[LightGBM] [Info] Number of data points in the train set: 25201, number of used features: 36
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.290068 -> initscore=-0.

c:\Users\USER\.conda\envs\deepLearning\lib\site-packages\lightgbm\engine.py:204: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")
c:\Users\USER\.conda\envs\deepLearning\lib\site-packages\lightgbm\engine.py:204: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")
c:\Users\USER\.conda\envs\deepLearning\lib\site-packages\lightgbm\engine.py:204: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


[LightGBM] [Info] Number of positive: 7310, number of negative: 17891
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001128 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3411
[LightGBM] [Info] Number of data points in the train set: 25201, number of used features: 36
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.290068 -> initscore=-0.895055
[LightGBM] [Info] Start training from score -0.895055
[LightGBM] [Info] Number of positive: 7310, number of negative: 17891
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000998 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3421
[LightGBM] [Info] Number of data points in the train set: 25201, number of used features: 36
[LightGBM] [Info] [b

c:\Users\USER\.conda\envs\deepLearning\lib\site-packages\lightgbm\engine.py:204: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")
c:\Users\USER\.conda\envs\deepLearning\lib\site-packages\lightgbm\engine.py:204: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")
c:\Users\USER\.conda\envs\deepLearning\lib\site-packages\lightgbm\engine.py:204: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


...

c:\Users\USER\.conda\envs\deepLearning\lib\site-packages\lightgbm\engine.py:204: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


[LightGBM] [Info] Number of positive: 7309, number of negative: 17892
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001619 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3414
[LightGBM] [Info] Number of data points in the train set: 25201, number of used features: 36
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.290028 -> initscore=-0.895247
[LightGBM] [Info] Start training from score -0.895247


c:\Users\USER\.conda\envs\deepLearning\lib\site-packages\numpy\ma\core.py:2820: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,
c:\Users\USER\.conda\envs\deepLearning\lib\site-packages\lightgbm\engine.py:204: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


[LightGBM] [Info] Number of positive: 9137, number of negative: 22364
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001917 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3447
[LightGBM] [Info] Number of data points in the train set: 31501, number of used features: 36
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.290054 -> initscore=-0.895120
[LightGBM] [Info] Start training from score -0.895120


GridSearchCV(cv=5, estimator=LGBMClassifier(random_state=0),
             param_grid={'max_depth': [10, 15, 20], 'num_iterations': [50, 100],
                         'reg_alpha': [0.0, 0.2, 0.3, 0.5, 0.7, 0.9],
                         'reg_lambda': [0.0, 0.2, 0.3, 0.5, 0.7, 0.9]})

In [42]:
grid_lgbm.cv_results_

{'mean_fit_time': array([0.09499969, 0.09340005, 0.09360042, 0.09760017, 0.09600129,
        0.09564571, 0.11080074, 0.10020065, 0.09900169, 0.10259995,
        0.10220051, 0.10660028, 0.10500126, 0.10860081, 0.11320024,
        0.10959959, 0.107199  , 0.12020068, 0.10839963, 0.13259888,
        0.15080223, 0.14560008, 0.14740009, 0.1457994 , 0.15144901,
        0.14759951, 0.14979968, 0.16425948, 0.14979973, 0.14819989,
        0.14380088, 0.14640036, 0.14680052, 0.15019984, 0.15380168,
        0.15065899, 0.23479939, 0.31960115, 0.33839955, 0.22319951,
        0.21419911, 0.24180055, 0.3996016 , 0.33460107, 0.25859828,
        0.24783773, 0.21320038, 0.2212007 , 0.21099997, 0.22259955,
        0.22200141, 0.22209682, 0.24380002, 0.24259863, 0.24779854,
        0.24860001, 0.24559917, 0.21700034, 0.21940031, 0.23500004,
        0.22519851, 0.22079954, 0.23020091, 0.2365993 , 0.22640052,
        0.22660108, 0.22559958, 0.23180046, 0.22942739, 0.22479973,
        0.24099975, 0.2224    ,

In [43]:
index_for_best_score = grid_lgbm.cv_results_.get('rank_test_score').argmin()
grid_lgbm.cv_results_.get('params')[index_for_best_score]

{'max_depth': 15, 'num_iterations': 100, 'reg_alpha': 0.9, 'reg_lambda': 0.5}

In [45]:
grid_lgbm.cv_results_.get('mean_test_score')[index_for_best_score]

0.7226755642213506

In [44]:
index_for_best_score

141

In [64]:
best = grid_lgbm.best_estimator_

In [67]:
(best.predict(dt_v.drop(columns='Churn')) == dt_v.Churn).mean()

0.7215021109547441

In [ ]:
#StratifiedKFold

In [48]:
sifK = StratifiedKFold()

In [52]:
results = {}

for i,(train_index,test_index) in enumerate(sifK.split(dt_t, dt_t.Churn)):
    temp = dt_t.iloc[train_index]
    train, valid = train_test_split(temp, test_size= len(dt_v)/len(temp) )
    tset = dt_t.iloc[test_index]

    print(f'{i+1}-fold')
    display(train['Churn'].value_counts()/len(train.Churn))
    display(valid.Churn.value_counts()/len(valid.Churn))
    display(dt_v.Churn.value_counts()/len(dt_v.Churn))

1-fold


Churn
0    0.707838
1    0.292162
Name: count, dtype: float64

Churn
0    0.711799
1    0.288201
Name: count, dtype: float64

Churn
0    0.71091
1    0.28909
Name: count, dtype: float64

2-fold


...

Churn
0    0.71091
1    0.28909
Name: count, dtype: float64

5-fold


Churn
0    0.712137
1    0.287863
Name: count, dtype: float64

Churn
0    0.708096
1    0.291904
Name: count, dtype: float64

Churn
0    0.71091
1    0.28909
Name: count, dtype: float64

In [61]:
results = {}

for i,(train_index,test_index) in enumerate(sifK.split(dt_t,dt_t.Churn)):
    temp = dt_t.iloc[train_index]
    train, valid = train_test_split(temp, test_size= len(dt_v)/len(temp) )
    tset = dt_t.iloc[test_index]

    lgbm = LGBMClassifier(max_depth = 15, num_iterations =  100, reg_alpha = 0.9, reg_lambda = 0.5, random_state=seed)
    lgbm.fit(train.drop(columns='Churn'),train.Churn,
         eval_set=(valid.drop(columns='Churn'),valid.Churn))
    
    
    confusion_matrix_res =confusion_matrix(
        tset.Churn,
        lgbm.predict(tset.drop(columns='Churn'),),labels=[1,0]
    )

    TP = confusion_matrix_res[0,0] 
    FN = confusion_matrix_res[0,1] 
    FP = confusion_matrix_res[1,0] 
    TN = confusion_matrix_res[1,1] 

    
    precision = TP/(TP+FP)
    recall = TP/(TP+FN)
    f1 = 2*precision*recall/(precision+recall)
    acc = (TP+TN)/(TP+TN+FP+FN)
    results.update({
        f'fold({i+1})' : {
            'precision' : precision,
            'recall' : recall,
            'f1' : f1 ,
            'acc' : acc
        }})
    print( f'fold {i+1}| precision : {precision}, recall : {recall} , f1 : {f1} ,  acc : {acc}' )
    

c:\Users\USER\.conda\envs\deepLearning\lib\site-packages\lightgbm\engine.py:204: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")
c:\Users\USER\.conda\envs\deepLearning\lib\site-packages\lightgbm\engine.py:204: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


[LightGBM] [Info] Number of positive: 3382, number of negative: 8317
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001620 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3194
[LightGBM] [Info] Number of data points in the train set: 11699, number of used features: 36
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.289085 -> initscore=-0.899834
[LightGBM] [Info] Start training from score -0.899834
fold 1| precision : 0.5290697674418605, recall : 0.14934354485776805 , f1 : 0.23293515358361777 ,  acc : 0.714648468497064
[LightGBM] [Info] Number of positive: 3470, number of negative: 8230
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000602 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3182
[LightGBM] [Info] Number of data points in the train set

c:\Users\USER\.conda\envs\deepLearning\lib\site-packages\lightgbm\engine.py:204: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")
c:\Users\USER\.conda\envs\deepLearning\lib\site-packages\lightgbm\engine.py:204: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


fold 4| precision : 0.5075921908893709, recall : 0.12807881773399016 , f1 : 0.20454545454545456 ,  acc : 0.7111111111111111
[LightGBM] [Info] Number of positive: 3377, number of negative: 8323
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001584 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3109
[LightGBM] [Info] Number of data points in the train set: 11700, number of used features: 36
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.288632 -> initscore=-0.902035
[LightGBM] [Info] Start training from score -0.902035
fold 5| precision : 0.5247058823529411, recall : 0.12199124726477024 , f1 : 0.1979582778517532 ,  acc : 0.7131746031746031


c:\Users\USER\.conda\envs\deepLearning\lib\site-packages\lightgbm\engine.py:204: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


In [63]:
for fold,res in results.items():
    print(f'{fold}: {res}')

fold(1): {'precision': 0.5290697674418605, 'recall': 0.14934354485776805, 'f1': 0.23293515358361777, 'acc': 0.714648468497064}
fold(2): {'precision': 0.5532359081419624, 'recall': 0.14504652435686918, 'f1': 0.22983521248915872, 'acc': 0.7180952380952381}
fold(3): {'precision': 0.5525114155251142, 'recall': 0.1324575807334428, 'f1': 0.21368653421633554, 'acc': 0.7173015873015873}
fold(4): {'precision': 0.5075921908893709, 'recall': 0.12807881773399016, 'f1': 0.20454545454545456, 'acc': 0.7111111111111111}
fold(5): {'precision': 0.5247058823529411, 'recall': 0.12199124726477024, 'f1': 0.1979582778517532, 'acc': 0.7131746031746031}
